### Creating and Persisting an ML Model

In [590]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [591]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [592]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

In [593]:

# Initial version
include = ['health', 'absences','age']

# include = ['health', 'absences', 'failures', 'Medu', 'Fedu', 'G3']
# include = ['absences', 'studytime', 'failures']
# include = ['absences', 'studytime', 'failures', 'activities', 'freetime']

include.append('G3')

df.drop(columns=df.columns.difference(include), inplace=True) 

In [594]:
df.info

<bound method DataFrame.info of      age  health  absences  G3
0     18       3         6   6
1     17       3         4   6
2     15       3        10  10
3     15       5         2  15
4     16       5         4  10
..   ...     ...       ...  ..
390   20       4        11   9
391   17       2         3  16
392   21       3         3   7
393   18       5         0  10
394   19       5         5   9

[395 rows x 4 columns]>

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [595]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [596]:
df.describe()

,age,health,absences,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,3.554430,5.708861,10.415190,0.184810
std,1.276043,1.390303,8.003096,4.581443,0.388636
min,15.000000,1.000000,0.000000,0.000000,0.000000
25%,16.000000,3.000000,0.000000,8.000000,0.000000
50%,17.000000,4.000000,4.000000,11.000000,0.000000
75%,18.000000,5.000000,8.000000,14.000000,0.000000
max,22.000000,5.000000,75.000000,20.000000,1.000000


Drop the G3 score

In [597]:

# Initial Version
include = ['health', 'absences','age']

# include = ['health', 'absences', 'failures', 'Medu', 'Fedu']
# include = ['absences', 'studytime', 'failures']
# include = ['absences', 'studytime', 'failures', 'activities', 'freetime']

include.append('qual_student')

df.drop(columns=df.columns.difference(include), inplace=True) 
df.describe()

,age,health,absences,qual_student
count,395.000000,395.000000,395.000000,395.000000
mean,16.696203,3.554430,5.708861,0.184810
std,1.276043,1.390303,8.003096,0.388636
min,15.000000,1.000000,0.000000,0.000000
25%,16.000000,3.000000,0.000000,0.000000
50%,17.000000,4.000000,4.000000,0.000000
75%,18.000000,5.000000,8.000000,0.000000
max,22.000000,5.000000,75.000000,1.000000


Import scikit-learn and build a random forest classifer

In [598]:
import sklearn
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split
dependent_variable = 'qual_student'

df = pd.get_dummies(df)


x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
clf = rf(n_estimators = 1000)
clf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=1000)

Run the prediction with the training data

In [599]:
pred = clf.predict(x_train)

print('****Results****')
print("Accuracy: {:.4%}".format(clf.score(x_train, y_train)))
print("F1 Score: {:.4%}".format(sklearn.metrics.f1_score(y_train,pred, average='binary')))
print("Recall Score: {:.4%}".format(sklearn.metrics.recall_score(y_train,pred, average='binary')))
print("Precision Score: {:.4%}".format(sklearn.metrics.precision_score(y_train,pred, average='binary')))

****Results****
Accuracy: 88.4058%
F1 Score: 64.4444%
Recall Score: 53.7037%
Precision Score: 80.5556%


Run the prediction with the testing data

In [600]:
pred = clf.predict(x_test)
sklearn.metrics.f1_score(y_test, pred, average='binary')

print('****Results****')
print("Accuracy: {:.4%}".format(clf.score(x_test, y_test)))
print("F1 Score: {:.4%}".format(sklearn.metrics.f1_score(y_test,pred, average='binary')))
print("Recall Score: {:.4%}".format(sklearn.metrics.recall_score(y_test,pred, average='binary')))
print("Precision Score: {:.4%}".format(sklearn.metrics.precision_score(y_test,pred, average='binary')))

****Results****
Accuracy: 76.4706%
F1 Score: 12.5000%
Recall Score: 10.5263%
Precision Score: 15.3846%


In [601]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, 'dockerfile/apps/model2.pkl')

['dockerfile/apps/model2.pkl']

In [602]:
# query_df = pd.DataFrame({ 'age' : pd.Series(1) ,'health' : pd.Series(15) ,'absences' : pd.Series(10)})
# query_df = pd.DataFrame({ 'health' : pd.Series(15), 'absences' : pd.Series(5), 'Dalc' : pd.Series(10), 'Walc' : pd.Series(10)})

query_df = pd.DataFrame({ 'health' : pd.Series(15), 'absences' : pd.Series(5), 'failures' : pd.Series(10), 'Medu' : pd.Series(3), 'Fedu' : pd.Series(4),})

In [603]:
pred = clf.predict(query_df)

/usr/local/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Fedu
- Medu
- failures
Feature names seen at fit time, yet now missing:
- age

  warnings.warn(message, FutureWarning)


ValueError: X has 5 features, but RandomForestClassifier is expecting 3 features as input.

In [ ]:
x

In [ ]:
type(x)